# FarmWise: Farmland Segmentation and Size Classification with U-Net

**Date**: April 14, 2025

This notebook implements a farm segmentation system using U-Net architecture to identify agricultural fields from satellite imagery, calculate their sizes, and classify them for targeted recommendations.

## Project Overview

**Goal**: Create a system that can:
1. Detect and segment farmlands from satellite imagery
2. Calculate the size/area of each identified farm
3. Classify farms by size (small, medium, large)
4. Enable a recommendation system based on farm size classification

**Approach**: U-Net architecture for semantic segmentation

## 1. Business Understanding

### 1.1 Problem Statement

Agricultural recommendations are most effective when tailored to the specific context of a farm, with farm size being a crucial factor. Large farms may benefit from different techniques, equipment, and crop selections compared to small ones. This project aims to automatically classify farms by size from satellite imagery to enable targeted recommendations.

### 1.2 Success Criteria

- **Technical Success**: Achieve high accuracy in farmland segmentation (IoU > 0.75)
- **Business Success**: Enable accurate size-based classification of farms for targeted recommendations

## 2. Data Acquisition and Understanding

### 2.1 Setup and Environment Preparation

In [ ]:
# Check for Kaggle environment and set up dependencies for GPU acceleration
!pip install torch torchvision matplotlib numpy pillow scikit-learn scikit-image opencv-python roboflow tqdm

: 

: 

In [ ]:
# Import required libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from sklearn.model_selection import train_test_split
import cv2
from skimage import measure
from tqdm.notebook import tqdm
from roboflow import Roboflow

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

CUDA Available: False
No GPU available, using CPU. This will be slower.


### 2.2 Data Acquisition from Roboflow

In [ ]:
# Initialize Roboflow and load dataset
# Note: You will need to provide your Roboflow API key
rf = Roboflow(api_key="HE9CEH5JxJ3U0vXrQTOy")  # Replace with your actual API key
project = rf.workspace("sid-mp92l").project("final-detectron-2")
dataset = project.version(1).download("yolov8")

# Print dataset path
print(f"Dataset downloaded to: {dataset.location}")

loading Roboflow workspace...
loading Roboflow project...
loading Roboflow project...
Dataset downloaded to: /kaggle/working/Final-Detectron-2-1
Dataset downloaded to: /kaggle/working/Final-Detectron-2-1


### 2.3 Dataset Exploration

In [ ]:
# Explore the dataset structure
def explore_directory(path, level=0):
    print('  ' * level + f"|-- {os.path.basename(path)}")
    if os.path.isdir(path):
        for item in os.listdir(path)[:10]:  # Limit to first 10 items
            item_path = os.path.join(path, item)
            if os.path.isdir(item_path):
                explore_directory(item_path, level + 1)
            else:
                print('  ' * (level + 1) + f"|-- {item}")
        if len(os.listdir(path)) > 10:
            print('  ' * (level + 1) + f"|-- ... ({len(os.listdir(path)) - 10} more items)")

print("Dataset Structure:")
explore_directory(dataset.location)

Dataset Structure:
|-- Final-Detectron-2-1
  |-- data.yaml
  |-- README.dataset.txt
  |-- valid
    |-- images
      |-- F_339_jpg.rf.51e2ccc5bf9110500c7c741dd02c850c.jpg
      |-- Image1302_jpg.rf.480c3275c79cf6f0fe5a12da7bc26de9.jpg
      |-- 0231133310001203_png_jpg.rf.8e459df1c3bb07a250e06f5466b4be06.jpg
      |-- tile_16_7_png_jpg.rf.2c2c4106ef0b72d1fe5c1c93f21628e3.jpg
      |-- F_231_jpg.rf.14d7b0a341204d5449770168be048bf6.jpg
      |-- Image1239_jpg.rf.f5e4174c2e4800c1cf34209fffe7c3ed.jpg
      |-- F_30002777_jpg.rf.4e10a4baa58cb9e36e45775e58e5ffeb.jpg
      |-- F_3000409_jpg.rf.0da484949be4a73e59e9a23d6af6a79c.jpg
      |-- F_30002029_jpg.rf.cd287ef949f58016c4af785cff991970.jpg
      |-- F_30002494_jpg.rf.17cf6a446e646d44fb036af064cd103e.jpg
      |-- ... (708 more items)
    |-- labels
      |-- F_30001820_jpg.rf.ba694da5df8a77c724abe5295c7dae02.txt
      |-- F_30002748_jpg.rf.2af9a581c54c84cd3eff25a73cbfb669.txt
      |-- 0303220111030101_png_jpg.rf.d2db1ac66b6db39fdc5674141

In [ ]:
# Visualize some sample images and masks
def visualize_samples(data_dir, num_samples=3):
    # Paths for train images and labels
    train_img_dir = os.path.join(data_dir, 'train', 'images')
    train_mask_dir = os.path.join(data_dir, 'train', 'labels')
    
    img_files = os.listdir(train_img_dir)[:num_samples]
    
    # Read the data.yaml file to get class information
    yaml_path = os.path.join(data_dir, 'data.yaml')
    class_names = ['Unknown']
    if os.path.exists(yaml_path):
        try:
            import yaml
            with open(yaml_path, 'r') as f:
                data_yaml = yaml.safe_load(f)
                if 'names' in data_yaml:
                    class_names = data_yaml['names']
                    print(f"Classes found in dataset: {class_names}")
        except Exception as e:
            print(f"Error reading class names from data.yaml: {e}")
    
    fig, axes = plt.subplots(num_samples, 2, figsize=(12, 4*num_samples))
    
    for i, img_file in enumerate(img_files):
        # Load image
        img_path = os.path.join(train_img_dir, img_file)
        img = Image.open(img_path)
        
        # Find corresponding mask (YOLOv8 format)
        mask_file = os.path.splitext(img_file)[0] + '.txt'
        mask_path = os.path.join(train_mask_dir, mask_file)
        
        # For visualization, we'll create a binary mask from YOLOv8 annotations
        # This is simplified and will need to be adjusted based on actual format
        mask = np.zeros(img.size[::-1], dtype=np.uint8)
        
        if os.path.exists(mask_path):
            # Read YOLOv8 format annotations (class x_center y_center width height)
            with open(mask_path, 'r') as f:
                lines = f.readlines()
                
            img_width, img_height = img.size
            for line in lines:
                parts = line.strip().split(' ')
                if len(parts) >= 5:  # Basic format check
                    # Convert normalized coordinates to pixel values
                    class_id = int(parts[0])
                    class_name = class_names[class_id] if class_id < len(class_names) else f"Class {class_id}"
                    
                    # We're only interested in the "Farms" class (or all classes if we're not sure)
                    if "farm" in class_name.lower() or True:  # Currently process all classes
                        x_center = float(parts[1]) * img_width
                        y_center = float(parts[2]) * img_height
                        width = float(parts[3]) * img_width
                        height = float(parts[4]) * img_height
                        
                        # Check if we have polygon points (instance segmentation)
                        if len(parts) > 5:
                            # Extract polygon points
                            for j in range(5, len(parts), 2):
                                if j+1 < len(parts):
                                    x = float(parts[j]) * img_width
                                    y = float(parts[j+1]) * img_height
                                    polygon_points.append((x, y))
                            
                            if polygon_points:
                                # Convert to numpy array for OpenCV
                                pts = np.array(polygon_points, np.int32)
                                pts = pts.reshape((-1, 1, 2))
                                # Fill polygon with ones
                                cv2.fillPoly(mask, [pts], 255)
                        else:
                            # Calculate bounding box coordinates
                            x1 = int(x_center - width / 2)
                            y1 = int(y_center - height / 2)
                            x2 = int(x_center + width / 2)
                            y2 = int(y_center + height / 2)
                            
                            # Draw rectangle on mask
                            cv2.rectangle(mask, (x1, y1), (x2, y2), 255, -1)  # Fill rectangle
        
        # Display image and mask
        if num_samples == 1:
            axes[0].imshow(np.array(img))
            axes[0].set_title(f"Image: {img_file}")
            axes[0].axis('off')
            
            axes[1].imshow(mask, cmap='gray')
            axes[1].set_title(f"Mask (from YOLO annotations)")
            axes[1].axis('off')
        else:
            axes[i, 0].imshow(np.array(img))
            axes[i, 0].set_title(f"Image: {img_file}")
            axes[i, 0].axis('off')
            
            axes[i, 1].imshow(mask, cmap='gray')
            axes[i, 1].set_title(f"Mask (from YOLO annotations)")
            axes[i, 1].axis('off')
    
    plt.tight_layout()
    plt.show()

try:
    visualize_samples(dataset.location)
except Exception as e:
    print(f"Error visualizing samples: {e}")
    print("Note: Adjust the visualization code based on the actual data format you received.")

### 2.4 Data Preparation

We need to convert YOLOv8 format annotations to segmentation masks for U-Net training.

In [ ]:
# Create a custom dataset class to load images and generate masks
class FarmlandDataset(Dataset):
    def __init__(self, img_dir, mask_dir, transform=None, farm_class_id=None, img_size=256):
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.img_size = img_size
        self.img_files = sorted([f for f in os.listdir(img_dir) if f.endswith(('.png', '.jpg', '.jpeg'))])
        
        # Try to load class names from data.yaml
        self.class_names = ['Unknown']
        self.farm_class_id = farm_class_id  # If specified, only use this class ID for farm segmentation
        yaml_path = os.path.join(os.path.dirname(os.path.dirname(img_dir)), 'data.yaml')
        if os.path.exists(yaml_path):
            try:
                import yaml
                with open(yaml_path, 'r') as f:
                    data_yaml = yaml.safe_load(f)
                    if 'names' in data_yaml:
                        self.class_names = data_yaml['names']
                        # If farm_class_id is not provided, try to find "Farms" class automatically
                        if self.farm_class_id is None:
                            for i, name in enumerate(self.class_names):
                                if "farm" in name.lower():
                                    print(f"Found Farm class: {name} with ID {i}")
                                    self.farm_class_id = i
                                    break
            except Exception as e:
                print(f"Error reading class names from data.yaml: {e}")
    
    def __len__(self):
        return len(self.img_files)
    
    def __getitem__(self, idx):
        # Load image
        img_path = os.path.join(self.img_dir, self.img_files[idx])
        image = Image.open(img_path).convert("RGB")
        
        # Generate mask from YOLOv8 annotation
        mask_file = os.path.splitext(self.img_files[idx])[0] + '.txt'
        mask_path = os.path.join(self.mask_dir, mask_file)
        
        # Create empty mask with the same size as the image
        original_size = image.size[::-1]  # (height, width)
        mask = np.zeros(original_size, dtype=np.float32)
        
        if os.path.exists(mask_path):
            # Read YOLOv8 format annotations
            with open(mask_path, 'r') as f:
                lines = f.readlines()
            
            img_width, img_height = image.size
            for line in lines:
                parts = line.strip().split(' ')
                if len(parts) >= 5:  # Basic format check for bounding box
                    # Extract class ID
                    class_id = int(parts[0])
                    
                    # Only process the farm class if specified, otherwise process all classes
                    if self.farm_class_id is None or class_id == self.farm_class_id:
                        # Check if we have polygon points (instance segmentation)
                        if len(parts) > 5:
                            # Extract polygon points
                            polygon_points = []
                            for i in range(5, len(parts), 2):
                                if i+1 < len(parts):
                                    x = float(parts[i]) * img_width
                                    y = float(parts[i+1]) * img_height
                                    polygon_points.append((x, y))
                            
                            if polygon_points:
                                # Convert to numpy array for OpenCV
                                pts = np.array(polygon_points, np.int32)
                                pts = pts.reshape((-1, 1, 2))
                                # Fill polygon with ones
                                cv2.fillPoly(mask, [pts], 1)
                        else:
                            # Use bounding box if no polygon points
                            x_center = float(parts[1]) * img_width
                            y_center = float(parts[2]) * img_height
                            width = float(parts[3]) * img_width
                            height = float(parts[4]) * img_height
                            
                            x1 = int(x_center - width / 2)
                            y1 = int(y_center - height / 2)
                            x2 = int(x_center + width / 2)
                            y2 = int(y_center + height / 2)
                            
                            cv2.rectangle(mask, (x1, y1), (x2, y2), 1, -1)
        
        # Create a custom transformation for the mask to ensure it's the same size as the transformed image
        if self.transform:
            # First resize the mask to the target size
            resized_mask = cv2.resize(mask, (self.img_size, self.img_size), interpolation=cv2.INTER_NEAREST)
            
            # Apply transforms to image
            image_tensor = self.transform(image)
            
            # Convert mask to tensor (without normalization)
            mask_tensor = torch.from_numpy(resized_mask).float().unsqueeze(0)
            
            # Double-check that dimensions match
            if image_tensor.shape[-2:] != mask_tensor.shape[-2:]:
                # If dimensions still don't match, force resize the mask to match exactly
                mask_tensor = F.interpolate(
                    mask_tensor.unsqueeze(0), 
                    size=image_tensor.shape[-2:], 
                    mode='nearest'
                ).squeeze(0)
        else:
            # If no transform is applied, resize both to a fixed size and convert to tensor
            image = transforms.Resize((self.img_size, self.img_size))(image)
            image_tensor = transforms.ToTensor()(image)
            
            resized_mask = cv2.resize(mask, (self.img_size, self.img_size), interpolation=cv2.INTER_NEAREST)
            mask_tensor = torch.from_numpy(resized_mask).float().unsqueeze(0)
        
        # Print shapes for debugging (only for the first few items)
        if idx < 3:  
            print(f"Image shape: {image_tensor.shape}, Mask shape: {mask_tensor.shape}")
            
        return image_tensor, mask_tensor

### 2.5 Dataset Organization and Augmentation

In [ ]:
## 2.5 Dataset Organization and Augmentation

# We'll create a more robust data pipeline with strong augmentation to improve model generalization
print("Setting up improved data organization and augmentation pipeline...")

# Import additional libraries for augmentation
!pip install -q albumentations
import albumentations as A
from albumentations.pytorch import ToTensorV2
import random
from sklearn.model_selection import KFold

# Define a function to organize the dataset more systematically
def organize_dataset(dataset_location, val_split=0.2, test_split=0.1, seed=42):
    """
    Organize dataset into properly stratified train/val/test splits
    """
    # Set random seed for reproducibility
    random.seed(seed)
    np.random.seed(seed)
    
    # Identify all image files
    train_img_dir = os.path.join(dataset_location, 'train', 'images')
    val_img_dir = os.path.join(dataset_location, 'valid', 'images')
    
    train_files = [f for f in os.listdir(train_img_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
    val_files = [f for f in os.listdir(val_img_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
    
    print(f"Found {len(train_files)} training images and {len(val_files)} validation images")
    
    # If validation set already exists and is reasonable size, use it
    if len(val_files) > 0 and len(val_files) >= len(train_files) * 0.1:
        print(f"Using existing validation split with {len(val_files)} images")
        return {
            'train': train_files,
            'val': val_files,
            'test': val_files[:max(1, int(len(val_files) * 0.5))]  # Use part of validation as test set
        }
    
    # Otherwise, create our own splits
    all_files = train_files + val_files
    np.random.shuffle(all_files)
    
    # Calculate split sizes
    test_size = max(1, int(len(all_files) * test_split))
    val_size = max(1, int(len(all_files) * val_split))
    train_size = len(all_files) - val_size - test_size
    
    # Create splits
    train_files = all_files[:train_size]
    val_files = all_files[train_size:train_size+val_size]
    test_files = all_files[train_size+val_size:]
    
    print(f"Created new splits: {len(train_files)} training, {len(val_files)} validation, {len(test_files)} test images")
    
    return {
        'train': train_files,
        'val': val_files,
        'test': test_files
    }

# Define robust augmentation pipelines for training and validation
def get_training_augmentation(img_size=256):
    """
    Returns a strong augmentation pipeline for training to improve generalization
    """
    return A.Compose([
        A.RandomResizedCrop(height=img_size, width=img_size, scale=(0.8, 1.0)),
        A.Flip(p=0.5),
        A.RandomRotate90(p=0.5),
        A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=15, p=0.5),
        A.OneOf([
            A.ElasticTransform(alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03, p=0.5),
            A.GridDistortion(p=0.5),
            A.OpticalDistortion(distort_limit=1.0, shift_limit=0.5, p=0.5),
        ], p=0.3),
        A.OneOf([
            A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
            A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.5),
            A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
        ], p=0.3),
        A.GaussNoise(var_limit=(10.0, 50.0), p=0.2),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

def get_validation_augmentation(img_size=256):
    """
    Returns minimal augmentation for validation (just resizing and normalization)
    """
    return A.Compose([
        A.Resize(height=img_size, width=img_size),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

# Create an improved FarmlandDataset class with better augmentation handling
class ImprovedFarmlandDataset(Dataset):
    def __init__(self, dataset_location, file_names, transform=None, farm_class_id=None, img_size=256, mode='train'):
        self.dataset_location = dataset_location
        self.file_names = file_names
        self.transform = transform
        self.img_size = img_size
        self.mode = mode
        
        # Determine the image and mask directories based on mode
        if mode == 'train':
            self.img_dir = os.path.join(dataset_location, 'train', 'images')
            self.mask_dir = os.path.join(dataset_location, 'train', 'labels')
        else:  # val or test
            self.img_dir = os.path.join(dataset_location, 'valid', 'images')
            self.mask_dir = os.path.join(dataset_location, 'valid', 'labels')
        
        # Try to load class names from data.yaml
        self.class_names = ['Unknown']
        self.farm_class_id = farm_class_id
        yaml_path = os.path.join(dataset_location, 'data.yaml')
        
        if os.path.exists(yaml_path):
            try:
                import yaml
                with open(yaml_path, 'r') as f:
                    data_yaml = yaml.safe_load(f)
                    if 'names' in data_yaml:
                        self.class_names = data_yaml['names']
                        # If farm_class_id is not provided, try to find "Farms" class automatically
                        if self.farm_class_id is None:
                            for i, name in enumerate(self.class_names):
                                if "farm" in name.lower():
                                    print(f"Found Farm class: {name} with ID {i}")
                                    self.farm_class_id = i
                                    break
            except Exception as e:
                print(f"Error reading class names from data.yaml: {e}")
    
    def __len__(self):
        return len(self.file_names)
    
    def __getitem__(self, idx):
        # Get file name
        img_file = self.file_names[idx]
        
        # Ensure the file exists in the directory
        img_path = os.path.join(self.img_dir, img_file)
        if not os.path.exists(img_path):
            # If not, it might be in the other directory
            alt_img_dir = os.path.join(self.dataset_location, 'valid' if self.mode == 'train' else 'train', 'images')
            img_path = os.path.join(alt_img_dir, img_file)
            
            # Update mask directory too
            alt_mask_dir = os.path.join(self.dataset_location, 'valid' if self.mode == 'train' else 'train', 'labels')
            self.mask_dir = alt_mask_dir
        
        # Load image
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Generate mask from YOLOv8 annotation
        mask_file = os.path.splitext(img_file)[0] + '.txt'
        mask_path = os.path.join(self.mask_dir, mask_file)
        
        # Create empty mask with the same size as the image
        mask = np.zeros(image.shape[:2], dtype=np.float32)
        
        if os.path.exists(mask_path):
            # Read YOLOv8 format annotations
            with open(mask_path, 'r') as f:
                lines = f.readlines()
            
            img_height, img_width = image.shape[:2]
            for line in lines:
                parts = line.strip().split(' ')
                if len(parts) >= 5:  # Basic format check for bounding box
                    # Extract class ID
                    class_id = int(parts[0])
                    
                    # Only process the farm class if specified, otherwise process all classes
                    if self.farm_class_id is None or class_id == self.farm_class_id:
                        # Check if we have polygon points (instance segmentation)
                        if len(parts) > 5:
                            # Extract polygon points
                            polygon_points = []
                            for i in range(5, len(parts), 2):
                                if i+1 < len(parts):
                                    x = float(parts[i]) * img_width
                                    y = float(parts[i+1]) * img_height
                                    polygon_points.append((x, y))
                            
                            if polygon_points:
                                # Convert to numpy array for OpenCV
                                pts = np.array(polygon_points, np.int32)
                                pts = pts.reshape((-1, 1, 2))
                                # Fill polygon with ones
                                cv2.fillPoly(mask, [pts], 1)
                        else:
                            # Use bounding box if no polygon points
                            x_center = float(parts[1]) * img_width
                            y_center = float(parts[2]) * img_height
                            width = float(parts[3]) * img_width
                            height = float(parts[4]) * img_height
                            
                            x1 = int(x_center - width / 2)
                            y1 = int(y_center - height / 2)
                            x2 = int(x_center + width / 2)
                            y2 = int(y_center + height / 2)
                            
                            cv2.rectangle(mask, (x1, y1), (x2, y2), 1, -1)
        
        # Apply transformations using albumentations
        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image_tensor = augmented['image']
            mask_tensor = augmented['mask']
            
            # Ensure mask is single channel
            if len(mask_tensor.shape) == 2:
                mask_tensor = mask_tensor.unsqueeze(0)
        else:
            # Fallback if no transform is provided
            image = cv2.resize(image, (self.img_size, self.img_size))
            mask = cv2.resize(mask, (self.img_size, self.img_size), interpolation=cv2.INTER_NEAREST)
            
            # Convert to tensor
            image_tensor = torch.from_numpy(image.transpose(2, 0, 1)).float() / 255.0
            mask_tensor = torch.from_numpy(mask).float().unsqueeze(0)
        
        # Verify shapes for debugging
        if idx < 3 and self.mode == 'train':
            print(f"{self.mode} - Image shape: {image_tensor.shape}, Mask shape: {mask_tensor.shape}")
            
        return image_tensor, mask_tensor

# Organize dataset and create splits
splits = organize_dataset(dataset.location)

# Create datasets with appropriate augmentations
train_dataset = ImprovedFarmlandDataset(
    dataset_location=dataset.location,
    file_names=splits['train'],
    transform=get_training_augmentation(img_size=256),
    mode='train'
)

val_dataset = ImprovedFarmlandDataset(
    dataset_location=dataset.location, 
    file_names=splits['val'],
    transform=get_validation_augmentation(img_size=256),
    mode='val'
)

test_dataset = ImprovedFarmlandDataset(
    dataset_location=dataset.location,
    file_names=splits['test'],
    transform=get_validation_augmentation(img_size=256),
    mode='val'
)

print(f"Created augmented datasets:")
print(f"  - Training set: {len(train_dataset)} images")
print(f"  - Validation set: {len(val_dataset)} images") 
print(f"  - Test set: {len(test_dataset)} images")

# Visualize some augmented samples to verify the pipeline
def show_augmented_samples(dataset, num_samples=3, figsize=(18, 12)):
    """Show augmented samples from the dataset"""
    fig, axes = plt.subplots(num_samples, 2, figsize=figsize)
    
    for i in range(num_samples):
        idx = np.random.randint(0, len(dataset))
        image, mask = dataset[idx]
        
        # Convert tensors to numpy arrays for visualization
        image_np = image.permute(1, 2, 0).cpu().numpy()
        mask_np = mask.squeeze().cpu().numpy()
        
        # De-normalize image for better visualization
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        image_np = (image_np * std + mean)
        image_np = np.clip(image_np, 0, 1)
        
        # Display
        axes[i, 0].imshow(image_np)
        axes[i, 0].set_title(f"Augmented Image {i+1}")
        axes[i, 0].axis("off")
        
        axes[i, 1].imshow(mask_np, cmap='gray')
        axes[i, 1].set_title(f"Corresponding Mask {i+1}")
        axes[i, 1].axis("off")
    
    plt.tight_layout()
    plt.show()

# Visualize some augmented training samples
print("Visualizing augmented training samples:")
show_augmented_samples(train_dataset, num_samples=3)

# Update data loaders with the improved datasets
train_loader = DataLoader(
    train_dataset, 
    batch_size=batch_size, 
    shuffle=True, 
    num_workers=min(os.cpu_count(), 8),
    pin_memory=True,
    prefetch_factor=2,
    persistent_workers=True,
    drop_last=True  # Drop last incomplete batch for more stable training
)

val_loader = DataLoader(
    val_dataset, 
    batch_size=batch_size, 
    shuffle=False, 
    num_workers=min(os.cpu_count(), 4),
    pin_memory=True
)

test_loader = DataLoader(
    test_dataset, 
    batch_size=batch_size, 
    shuffle=False, 
    num_workers=min(os.cpu_count(), 4),
    pin_memory=True
)

## 3.2 Hyperparameter Tuning and Anti-Overfitting Techniques

In [ ]:
# Implement hyperparameter tuning and regularization to optimize the model
print("Setting up hyperparameter tuning and anti-overfitting techniques...")

# Import additional libraries for hyperparameter tuning
!pip install -q optuna
import optuna
from functools import partial
import copy

# Define regularization techniques to prevent overfitting
class UNetWithRegularization(nn.Module):
    def __init__(self, n_channels=3, n_classes=1, dropout_rate=0.2, use_dropout=True):
        super(UNetWithRegularization, self).__init__()
        self.use_dropout = use_dropout
        self.dropout_rate = dropout_rate
        
        # Encoder (downsampling)
        self.enc1 = DoubleConv(n_channels, 64)
        self.pool1 = nn.MaxPool2d(2)
        self.enc2 = DoubleConv(64, 128)
        self.pool2 = nn.MaxPool2d(2)
        self.enc3 = DoubleConv(128, 256)
        self.pool3 = nn.MaxPool2d(2)
        self.enc4 = DoubleConv(256, 512)
        self.pool4 = nn.MaxPool2d(2)
        
        # Dropout for bottleneck (most important for preventing overfitting)
        self.dropout = nn.Dropout2d(p=self.dropout_rate) if use_dropout else nn.Identity()
        
        # Bottleneck
        self.bottleneck = DoubleConv(512, 1024)
        
        # Decoder (upsampling)
        self.upconv4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.dec4 = DoubleConv(1024, 512)  # 512 + 512 input channels from skip connection
        self.upconv3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.dec3 = DoubleConv(512, 256)   # 256 + 256 input channels from skip connection
        self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec2 = DoubleConv(256, 128)   # 128 + 128 input channels from skip connection
        self.upconv1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec1 = DoubleConv(128, 64)    # 64 + 64 input channels from skip connection
        
        # Final output layer
        self.outconv = nn.Conv2d(64, n_classes, kernel_size=1)
        
    def forward(self, x):
        # Encoder
        e1 = self.enc1(x)
        p1 = self.pool1(e1)
        
        e2 = self.enc2(p1)
        p2 = self.pool2(e2)
        
        e3 = self.enc3(p2)
        p3 = self.pool3(e3)
        
        e4 = self.enc4(p3)
        p4 = self.pool4(e4)
        
        # Bottleneck with dropout
        p4 = self.dropout(p4)
        b = self.bottleneck(p4)
        
        # Decoder with skip connections
        u4 = self.upconv4(b)
        u4 = torch.cat([u4, e4], dim=1)  # Skip connection
        d4 = self.dec4(u4)
        
        u3 = self.upconv3(d4)
        u3 = torch.cat([u3, e3], dim=1)  # Skip connection
        d3 = self.dec3(u3)
        
        u2 = self.upconv2(d3)
        u2 = torch.cat([u2, e2], dim=1)  # Skip connection
        d2 = self.dec2(u2)
        
        u1 = self.upconv1(d2)
        u1 = torch.cat([u1, e1], dim=1)  # Skip connection
        d1 = self.dec1(u1)
        
        # Final output
        out = self.outconv(d1)
        return torch.sigmoid(out)  # Apply sigmoid for binary segmentation

# Define weighted loss function to handle class imbalance
class WeightedBCELoss(nn.Module):
    def __init__(self, pos_weight=2.0):
        super(WeightedBCELoss, self).__init__()
        self.pos_weight = pos_weight
        
    def forward(self, outputs, targets):
        # Apply weighted loss - higher weight for positive class (farmland)
        loss = F.binary_cross_entropy(outputs, targets, reduction='none')
        weights = targets * (self.pos_weight - 1.0) + 1.0
        weighted_loss = (loss * weights).mean()
        return weighted_loss

# Define combo loss (BCE + Dice loss) for better segmentation
class DiceBCELoss(nn.Module):
    def __init__(self, dice_weight=1.0, bce_weight=1.0):
        super(DiceBCELoss, self).__init__()
        self.dice_weight = dice_weight
        self.bce_weight = bce_weight
        self.bce_loss = nn.BCELoss()
        
    def forward(self, outputs, targets):
        # BCE Loss
        bce_loss = self.bce_loss(outputs, targets)
        
        # Dice Loss
        intersection = (outputs * targets).sum()
        dice_loss = 1 - (2. * intersection + 1.0) / (outputs.sum() + targets.sum() + 1.0)
        
        # Combined loss
        combo_loss = self.bce_weight * bce_loss + self.dice_weight * dice_loss
        return combo_loss

# Define the objective function for Optuna hyperparameter tuning
def objective(trial, max_epochs=10, device=device):
    # Define hyperparameters to tune
    params = {
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-2),
        'dropout_rate': trial.suggest_float('dropout_rate', 0.1, 0.5),
        'weight_decay': trial.suggest_loguniform('weight_decay', 1e-6, 1e-3),
        'batch_size': trial.suggest_categorical('batch_size', [8, 16, 32]),
        'loss_type': trial.suggest_categorical('loss_type', ['bce', 'weighted_bce', 'dice_bce']),
        'pos_weight': trial.suggest_float('pos_weight', 1.0, 5.0) if trial.params.get('loss_type', '') == 'weighted_bce' else 2.0,
        'dice_weight': trial.suggest_float('dice_weight', 0.5, 2.0) if trial.params.get('loss_type', '') == 'dice_bce' else 1.0,
    }
    
    # Create mini train/val datasets for faster tuning
    mini_train_size = min(len(train_dataset), 100)  # Use at most 100 samples for tuning
    mini_val_size = min(len(val_dataset), 50)       # Use at most 50 samples for validation
    
    indices_train = np.random.choice(len(train_dataset), mini_train_size, replace=False)
    indices_val = np.random.choice(len(val_dataset), mini_val_size, replace=False)
    
    mini_train_dataset = torch.utils.data.Subset(train_dataset, indices_train)
    mini_val_dataset = torch.utils.data.Subset(val_dataset, indices_val)
    
    # Create dataloaders with the selected batch size
    mini_train_loader = DataLoader(
        mini_train_dataset, 
        batch_size=params['batch_size'], 
        shuffle=True, 
        num_workers=4,
        pin_memory=True
    )
    
    mini_val_loader = DataLoader(
        mini_val_dataset, 
        batch_size=params['batch_size'], 
        shuffle=False, 
        num_workers=4,
        pin_memory=True
    )
    
    # Initialize model with regularization
    model = UNetWithRegularization(
        n_channels=3, 
        n_classes=1, 
        dropout_rate=params['dropout_rate'],
        use_dropout=True
    ).to(device)
    
    # Initialize optimizer with weight decay (L2 regularization)
    optimizer = optim.Adam(
        model.parameters(), 
        lr=params['learning_rate'],
        weight_decay=params['weight_decay']  # L2 regularization
    )
    
    # Initialize learning rate scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, 
        mode='min', 
        factor=0.5, 
        patience=3,
        verbose=True
    )
    
    # Initialize loss function based on hyperparameters
    if params['loss_type'] == 'weighted_bce':
        criterion = WeightedBCELoss(pos_weight=params['pos_weight'])
    elif params['loss_type'] == 'dice_bce':
        criterion = DiceBCELoss(dice_weight=params['dice_weight'], bce_weight=1.0)
    else:  # default: 'bce'
        criterion = nn.BCELoss()
    
    # Train for a few epochs
    best_val_loss = float('inf')
    
    for epoch in range(max_epochs):
        # Train
        model.train()
        train_loss = 0
        
        for images, masks in mini_train_loader:
            images = images.to(device)
            masks = masks.to(device)
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, masks)
            
            # Backward pass and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * images.size(0)
        
        train_loss /= len(mini_train_loader.dataset)
        
        # Validate
        model.eval()
        val_loss = 0
        
        with torch.no_grad():
            for images, masks in mini_val_loader:
                images = images.to(device)
                masks = masks.to(device)
                
                # Forward pass
                outputs = model(images)
                loss = criterion(outputs, masks)
                
                val_loss += loss.item() * images.size(0)
        
        val_loss /= len(mini_val_loader.dataset)
        
        # Update learning rate
        scheduler.step(val_loss)
        
        # Update best validation loss
        if val_loss < best_val_loss:
            best_val_loss = val_loss
        
        # Report intermediate objective value
        trial.report(val_loss, epoch)
        
        # Handle pruning based on the intermediate value
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
    
    return best_val_loss

# Function to run hyperparameter tuning
def run_hyperparameter_tuning(n_trials=20, max_epochs=10):
    print(f"Starting hyperparameter tuning with {n_trials} trials...")
    
    # Create a study object and optimize the objective function
    study = optuna.create_study(direction='minimize', pruner=optuna.pruners.MedianPruner())
    study.optimize(partial(objective, max_epochs=max_epochs, device=device), n_trials=n_trials)
    
    # Get the best hyperparameters
    best_params = study.best_params
    print(f"Best hyperparameters: {best_params}")
    print(f"Best validation loss: {study.best_value:.4f}")
    
    # Visualize the hyperparameter tuning results
    try:
        # Plot optimization history
        plt.figure(figsize=(10, 6))
        optuna.visualization.matplotlib.plot_optimization_history(study)
        plt.tight_layout()
        plt.show()
        
        # Plot parameter importances
        plt.figure(figsize=(10, 6))
        optuna.visualization.matplotlib.plot_param_importances(study)
        plt.tight_layout()
        plt.show()
    except:
        print("Could not generate Optuna visualization plots")
    
    return best_params

# Run a quick hyperparameter search with fewer trials for demonstration
best_params = run_hyperparameter_tuning(n_trials=5, max_epochs=5)

# Create the optimized model with the best hyperparameters
optimized_model = UNetWithRegularization(
    n_channels=3, 
    n_classes=1,
    dropout_rate=best_params.get('dropout_rate', 0.2),
    use_dropout=True
).to(device)

# Use DataParallel if multiple GPUs are available
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs for parallel training of optimized model")
    optimized_model = nn.DataParallel(optimized_model)

# Create the optimized loss function
if best_params.get('loss_type') == 'weighted_bce':
    criterion = WeightedBCELoss(pos_weight=best_params.get('pos_weight', 2.0))
elif best_params.get('loss_type') == 'dice_bce':
    criterion = DiceBCELoss(
        dice_weight=best_params.get('dice_weight', 1.0), 
        bce_weight=1.0
    )
else:
    criterion = nn.BCELoss()

# Create the optimized optimizer with weight decay for regularization
optimizer = optim.Adam(
    optimized_model.parameters(),
    lr=best_params.get('learning_rate', 0.001),
    weight_decay=best_params.get('weight_decay', 1e-4)  # L2 regularization
)

# Create learning rate scheduler for better convergence
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',
    factor=0.5,
    patience=5,
    verbose=True
)

# Print model and optimization summary
print("\nOptimized Model Configuration:")
print(f"- Dropout Rate: {best_params.get('dropout_rate', 0.2)}")
print(f"- Learning Rate: {best_params.get('learning_rate', 0.001)}")
print(f"- Weight Decay: {best_params.get('weight_decay', 1e-4)}")
print(f"- Loss Function: {best_params.get('loss_type', 'bce')}")
if best_params.get('loss_type') == 'weighted_bce':
    print(f"- Positive Class Weight: {best_params.get('pos_weight', 2.0)}")
elif best_params.get('loss_type') == 'dice_bce':
    print(f"- Dice Weight: {best_params.get('dice_weight', 1.0)}")
print(f"- Batch Size: {best_params.get('batch_size', 16)}")

## 3.2.1 Evaluation Metrics

In [ ]:
# Define evaluation metrics and functions
def calculate_iou(outputs, targets, threshold=0.5):
    """
    Calculate IoU (Intersection over Union) for segmentation masks
    Args:
        outputs: Model predictions (B, 1, H, W)
        targets: Ground truth masks (B, 1, H, W)
        threshold: Threshold to binarize predictions
    Returns:
        Average IoU across batch
    """
    # Binarize outputs at the given threshold
    outputs = (outputs > threshold).float()
    
    # Calculate intersection and union
    intersection = (outputs * targets).sum(dim=(1, 2, 3))  # Sum over HWC dimensions for each sample
    union = outputs.sum(dim=(1, 2, 3)) + targets.sum(dim=(1, 2, 3)) - intersection
    
    # Handle empty masks - set IoU to 1 if both prediction and target are empty
    iou = torch.zeros_like(intersection)
    valid = union > 0
    iou[valid] = intersection[valid] / union[valid]
    iou[~valid] = 1.0  # If both prediction and target are empty, IoU is 1
    
    # Return batch average
    return iou.mean().item()

def evaluate_model(model, dataloader, device, threshold=0.5):
    """Evaluate model on dataloader and return average IoU"""
    model.eval()
    iou_sum = 0.0
    count = 0
    
    with torch.no_grad():
        for images, masks in tqdm(dataloader, desc="Evaluating"):
            images = images.to(device)
            masks = masks.to(device)
            
            # Forward pass
            outputs = model(images)
            
            # Calculate IoU
            batch_iou = calculate_iou(outputs, masks, threshold)
            iou_sum += batch_iou * images.size(0)
            count += images.size(0)
    
    return iou_sum / count if count > 0 else 0.0

def visualize_predictions(model, dataloader, device, num_samples=3, threshold=0.5):
    """Visualize model predictions alongside ground truth masks"""
    model.eval()
    
    # Get a batch of images and masks
    images, masks = next(iter(dataloader))
    images = images.to(device)
    masks = masks.to(device)
    
    # Get predictions
    with torch.no_grad():
        outputs = model(images)
        binary_outputs = (outputs > threshold).float()
    
    # Visualize only a subset of the batch
    n = min(num_samples, images.size(0))
    fig, axes = plt.subplots(n, 3, figsize=(15, 5*n))
    
    for i in range(n):
        # Get image, mask, and prediction for this sample
        img = images[i].cpu()
        mask = masks[i].cpu()
        pred = outputs[i].cpu()
        binary_pred = binary_outputs[i].cpu()
        
        # Denormalize the image for visualization
        img = img.permute(1, 2, 0).numpy()
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        img = (img * std + mean)
        img = np.clip(img, 0, 1)
        
        # Convert masks to numpy
        mask = mask.squeeze().numpy()
        pred = pred.squeeze().numpy()
        binary_pred = binary_pred.squeeze().numpy()
        
        # Plot
        axes[i, 0].imshow(img)
        axes[i, 0].set_title("Input Image")
        axes[i, 0].axis("off")
        
        axes[i, 1].imshow(mask, cmap='gray')
        axes[i, 1].set_title("Ground Truth Mask")
        axes[i, 1].axis("off")
        
        axes[i, 2].imshow(binary_pred, cmap='gray')
        iou = calculate_iou(
            torch.tensor(pred).unsqueeze(0).unsqueeze(0), 
            torch.tensor(mask).unsqueeze(0).unsqueeze(0), 
            threshold
        )
        axes[i, 2].set_title(f"Prediction (IoU: {iou:.4f})")
        axes[i, 2].axis("off")
    
    plt.tight_layout()
    plt.show()
    
    return binary_outputs

## 3.3 Improved Training Loop with Anti-Overfitting Techniques

In [ ]:
# Train the model with the optimized hyperparameters and anti-overfitting measures
print("Starting training with optimized model and anti-overfitting techniques...")

# Define improved training functions with early stopping
def train_epoch_with_regularization(model, dataloader, criterion, optimizer, device, grad_clip=1.0):
    """Train for one epoch with gradient clipping to prevent exploding gradients"""
    model.train()
    running_loss = 0.0
    
    for images, masks in tqdm(dataloader, desc="Training"):
        images = images.to(device)
        masks = masks.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, masks)
        
        # Backward pass and optimize with gradient clipping
        optimizer.zero_grad()
        loss.backward()
        
        # Apply gradient clipping to prevent exploding gradients (another anti-overfitting technique)
        if grad_clip > 0:
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=grad_clip)
        
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
    
    epoch_loss = running_loss / len(dataloader.dataset)
    return epoch_loss

# Early stopping implementation
class EarlyStopping:
    def __init__(self, patience=7, min_delta=0, verbose=True):
        """
        Early stopping to stop training when validation loss doesn't improve
        Args:
            patience (int): How many epochs to wait after last improvement
            min_delta (float): Minimum change to qualify as improvement
            verbose (bool): If True, prints a message for each improvement
        """
        self.patience = patience
        self.min_delta = min_delta
        self.verbose = verbose
        self.counter = 0
        self.best_loss = float('inf')
        self.early_stop = False
        
    def __call__(self, val_loss, model, path):
        if val_loss < self.best_loss - self.min_delta:
            if self.verbose:
                print(f"Validation loss improved from {self.best_loss:.4f} to {val_loss:.4f}. Saving model...")
            self.best_loss = val_loss
            self.counter = 0
            torch.save(model.state_dict(), path)
        else:
            self.counter += 1
            if self.verbose:
                print(f"Validation loss did not improve. Counter: {self.counter}/{self.patience}")
            if self.counter >= self.patience:
                if self.verbose:
                    print("Early stopping triggered")
                self.early_stop = True
        
        return self.early_stop

# Define validation function
def validate_epoch(model, dataloader, criterion, device):
    """Validate the model for one epoch"""
    model.eval()
    running_loss = 0.0
    
    with torch.no_grad():
        for images, masks in tqdm(dataloader, desc="Validating"):
            images = images.to(device)
            masks = masks.to(device)
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, masks)
            
            running_loss += loss.item() * images.size(0)
    
    epoch_loss = running_loss / len(dataloader.dataset)
    return epoch_loss

# Set up training parameters
num_epochs = 50  # Maximum number of epochs
best_model_path = 'best_unet_regularized_model.pth'

# Set up early stopping
early_stopping = EarlyStopping(patience=10, verbose=True)

# Lists to store training history
train_losses = []
val_losses = []
val_ious = []
learning_rates = []

# Training loop with early stopping and regularization
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    
    # Get current learning rate
    current_lr = optimizer.param_groups[0]['lr']
    learning_rates.append(current_lr)
    print(f"Current learning rate: {current_lr:.7f}")
    
    # Train with gradient clipping
    train_loss = train_epoch_with_regularization(
        optimized_model, 
        train_loader, 
        criterion, 
        optimizer, 
        device,
        grad_clip=1.0  # Apply gradient clipping to prevent exploding gradients
    )
    train_losses.append(train_loss)
    
    # Validate
    val_loss = validate_epoch(optimized_model, val_loader, criterion, device)
    val_losses.append(val_loss)
    
    # Calculate IoU
    val_iou = evaluate_model(optimized_model, val_loader, device)
    val_ious.append(val_iou)
    
    # Print epoch results
    print(f"Train Loss: {train_loss:.4f}")
    print(f"Validation Loss: {val_loss:.4f}")
    print(f"Validation IoU: {val_iou:.4f}")
    
    # Update learning rate based on validation loss
    scheduler.step(val_loss)
    
    # Check early stopping
    if early_stopping(val_loss, optimized_model, best_model_path):
        print("Early stopping triggered")
        break
    
    print("-" * 50)

# Load the best model for evaluation
optimized_model.load_state_dict(torch.load(best_model_path))
print("Training completed with early stopping!")

# Final evaluation on test set
test_loss = validate_epoch(optimized_model, test_loader, criterion, device)
test_iou = evaluate_model(optimized_model, test_loader, device)

print(f"Final Test Loss: {test_loss:.4f}")
print(f"Final Test IoU: {test_iou:.4f}")

# Check if the model achieved the success criteria
if test_iou > 0.75:
    print("✅ Success! Model achieved the target IoU (> 0.75)")
else:
    print(f"Model achieved {test_iou:.4f} IoU, which is below the target of 0.75")